# How to create an Azure AI Content safety enabled LLaMA online endpoint
### This notebook will walk you through the steps to create an __Azure AI Content Safety__ enabled __LLaMA__ online endpoint.
### The steps are:
1. Create an __Azure AI Content Safety__ resource for moderating the request from user and response from the __LLaMA__ online endpoint.
2. Create a new __Azure AI Content Safety__ enabled __LLaMA__ online endpoint with a custom score.py which will integrate with the __Azure AI Content Safety__ resource to moderate the response from the __LLaMA__ model and the request from the user, but to make the custom score.py to sucessfully autheticated to the __Azure AI Content Safety__ resource, we have 2 options:
    1. __UAI__, recommended but more complex approach, is to create a __User Assigned Identity (UAI)__ and assign appropriate roles to the __UAI__. Then, the custom score.py can obtain the access token of the __UAI__ from the AAD server to access the Azure AI Content Safety resource.
    2. __Environment variable__, simpler but less secure approach, is to just pass the access key of the __Azure AI Content Safety__ resource to the custom score.py via environment variable, then the custom score.py can use the key directly to access the Azure AI Content Safety resource, this option is less secure than the first option, if someone in your org has access to the endpoint, he/she can get the access key from the environment variable and use it to access the Azure AI Content Safety resource.
  

### Please update below variable to EnvVar if you want to use the Environment variable approach.

In [ ]:
aacs_authentication_mode = "UAI"  # UAI or EnvVar, if EnvVar is used, the access keys of AACS and LLaMA online endpoint will be set to environment variables.

### 1. Prerequisites
#### 1.1 Check List:
- [x] A working Azure ML workspace with the default container registry created.
- [x] You have created an new Python virtual environment for this notebook.
- [x] The identity you are using to execute this notebook(your azure account or a Managed Identity attached to the VM) need to have the __Contributor__ role on the resource group where the AML Workspace your specified is located, because this notebook will create an Azure AI Content Safety resource using that identity.
- [x] __Required If you choose to use the UAI approach__, the identity executing this notebook (either yourself or your virtual machine) needs to have the __Owner__ role on the resource group that contains the specified AML Workspace. This is because the notebook will create a new UAI and assign the UAI some required roles to successfully create the Azure AI Content Safety enabled LLaMA endpoint.

#### 1.2 Install Dependencies

In [ ]:
%pip install azure-identity==1.13.0
%pip install azure-mgmt-cognitiveservices==13.4.0
%pip install azure-ai-ml==1.8.0
%pip install azure-mgmt-msi==7.0.0
%pip install azure-mgmt-authorization==3.0.0

#### 1.3 Assign variables for the workspace and deployment

In [ ]:
# NOTE: Update following workspace information to contain
#       your subscription ID, resource group name, and workspace name
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

#### 1.4 Decide on a name for your Content Safety enabled LLaMA online endpoint

In [ ]:
import random

rand = random.randint(0, 10000)

endpoint_name = (
    f"safetyllama{rand}"  # the final endpoint name of the safety enabled llama endpoint
)

# with the given name, there will be 4 resources created in the resource group of your AML workspace:
# 1. an Azure AI Content Safety resource: {endpoint_name}-aacs
# 2. [UAI approach]a UAI(User Assigned Identity): {endpoint_name}-uai
# 3. an Azure AI Content Safety enabled LLaMA online endpoint, which you will be using it to do your AI work: {endpoint_name}
aacs_name = f"{endpoint_name}-aacs"
uai_name = f"{endpoint_name}-uai"
print(f"going to create the following resources:")
print(f"-  Azure AI Content Safety resource: {aacs_name}")
if aacs_authentication_mode == "UAI":
    print(f"-  User Assigned Identity: {uai_name}")
print(f"-  Azure AI Content Safety enabled LLaMA online endpoint: {endpoint_name}")

if len(endpoint_name) > 32:
    raise Exception(
        f"the endpoint name {endpoint_name} is too long, please see: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-quotas?view=azureml-api-2#azure-machine-learning-managed-online-endpoints"
    )

### 2. Connect to your AML Workspace

In [ ]:
import os, json
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": workspace_name,
    }
    # write and reload from config file
    config_path = "./config.json"
    os.makedirs(os.path.dirname(config_path), exist_ok=True)
    with open(config_path, "w") as fo:
        fo.write(json.dumps(client_config))
    ml_client = MLClient.from_config(credential=credential, path=config_path)

workspace_location = ml_client.workspaces.get(ml_client.workspace_name).location
workspace_resource_id = ml_client.workspaces.get(ml_client.workspace_name).id
subscription_id = ml_client.subscription_id
resource_group_name = ml_client.resource_group_name
workspace_name = ml_client.workspace_name
print(f"Connected to workspace {workspace_resource_id}")
print(f"Workspace location is {workspace_location}")

### 3. Create Azure AI Content Safety

#### 3.1 Choose a region for your Azure AI Content Safety resource
__NOTE__: before you choose the region to deploy the Azure AI Content Safety, please aware of that your data will be transferred to the region you choose and by selecting a region outside your current location, you may be allowing the transmission of your data to regions outside your jurisdiction. It is important to note that data protection and privacy laws may vary between jurisdictions. Before proceeding, we strongly advise you to familiarize yourself with the local laws and regulations governing data transfer and ensure that you are legally permitted to transmit your data to an overseas location for processing. By continuing with the selection of a different region, you acknowledge that you have understood and accepted any potential risks associated with such data transmission. Please proceed with caution.

##### 3.1.1 List SKUs for Azure AI Content Safety

In [ ]:
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

client = CognitiveServicesManagementClient(credential, subscription_id)
print("Available SKUs:")
skus = client.resource_skus.list()
print("SKU Name\tSKU Tier\tLocations")
for sku in skus:
    if sku.kind == "ContentSafety":
        locations = ",".join(sku.locations)
        print(sku.name + "\t" + sku.tier + "\t" + locations)

##### 3.1.2 Choose a region and SKU for your Azure AI Content Safety resource

In [ ]:
aacs_location = "<AACS-Location>"
aacs_sku = "<AACS-SKU>"

print(f"will create AACS resource in {aacs_location} with sku {aacs_sku}")

#### 3.2 Create Azure AI Content Safety resource

In [ ]:
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.cognitiveservices.models import Account, Sku, AccountProperties
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import time

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

client = CognitiveServicesManagementClient(credential, subscription_id)
client.resource_skus.list()

# create a new Cognitive Services Account
kind = "ContentSafety"
parameters = Account(
    sku=Sku(name=aacs_sku),
    kind=kind,
    location=aacs_location,
    properties=AccountProperties(
        custom_sub_domain_name=aacs_name, public_network_access="Enabled"
    ),
)
# How many seconds to wait between checking the status of an async operation.
wait_time = 10

poller = client.accounts.begin_create(resource_group_name, aacs_name, parameters)
while not poller.done():
    print(
        "Waiting {wait_time} seconds for operation to finish.".format(
            wait_time=wait_time
        )
    )
    time.sleep(wait_time)
    # This will raise an exception if the server responded with an error.
    result = poller.result()

print("Resource created.")

aacs = client.accounts.get(resource_group_name, aacs_name)
aacs_endpoint = aacs.properties.endpoint
aacs_resource_id = aacs.id
print(f"AACS endpoint is {aacs_endpoint}")
print(f"AACS ResourceId is {aacs_resource_id}")

aacs_access_key = client.accounts.list_keys(
    resource_group_name=resource_group_name, account_name=aacs_name
).key1

### 4. [Reqiured for UAI approach] Create a Managed Identity for the Azure AI Content Safety enabled LLaMA endpoint

If you want to use the __EnvVar__ approach, you can skip this step and just get started from step 5.

#### 4.1 Create a Managed Identity for the Azure AI Content Safety enabled LLaMA endpoint
NOTE: Azure Content Safey is support AAD token based authentication by default, which means we need to create a new UAI for the Azure AI Content Safety enabled LLaMA endpoint, so that it can access the Azure AI Content Safety using AAD token.

##### 4.1.1 Get a handle to the ManagedServiceIdentityClient

In [ ]:
if aacs_authentication_mode == "UAI":
    from azure.mgmt.msi import ManagedServiceIdentityClient
    from azure.mgmt.msi.models import Identity

    msi_client = ManagedServiceIdentityClient(
        subscription_id=subscription_id,
        credential=credential,
    )

##### 4.1.2 Create the User Assigned Identity:

In [ ]:
if aacs_authentication_mode == "UAI":
    uai_name = f"{endpoint_name}-uai"
    print(
        f"Will create UAI(User Assigned Identity) {uai_name} for the Azure AI Content Safety enabled LLaMA endpoint."
    )

    msi_client.user_assigned_identities.create_or_update(
        resource_group_name=resource_group_name,
        resource_name=uai_name,
        parameters=Identity(location=workspace_location),
    )

##### 4.1.3 Retrieve the identity object
we need to retrieve the identity object so that we can use it to deploy the Azure AI Content Safety enabled LLaMA online endpoint.

In [ ]:
if aacs_authentication_mode == "UAI":
    uai_identity = msi_client.user_assigned_identities.get(
        resource_group_name=resource_group_name,
        resource_name=uai_name,
    )
    uai_principal_id = uai_identity.principal_id
    uai_client_id = uai_identity.client_id
    uai_id = uai_identity.id
    print(f"UAI principal id: {uai_principal_id}")
    print(f"UAI client id: {uai_client_id}")
    print(f"UAI id: {uai_id}")

#### 4.2 Grant appropriate roles to the UAI we created above.
Note: In order to successfully run scripts in current step, your must have owner permission on the AACS resource and the LLaMA endpoint, which we created in the previous steps.

##### 4.2.1 Get an AuthorizationManagementClient to list Role Definitions

In [ ]:
if aacs_authentication_mode == "UAI":
    from azure.mgmt.authorization import AuthorizationManagementClient
    from azure.mgmt.authorization.v2022_04_01.models import (
        RoleAssignmentCreateParameters,
    )
    import uuid

    role_definition_client = AuthorizationManagementClient(
        credential=credential,
        subscription_id=subscription_id,
        api_version="2022-04-01",
    )
    role_assignment_client = AuthorizationManagementClient(
        credential=credential,
        subscription_id=subscription_id,
        api_version="2022-04-01",
    )

    uai_role_check_list = {
        "Cognitive Services Contributor": {
            "step": "4.2.2",
            "description": "assigne the role Cognitive Services User to the UAI on the Azure AI Content Safety resource.",
        },
        "AcrPull": {
            "step": "4.2.3",
            "description": "assigne the role AcrPull to the UAI on the Azure Container Registry.",
        },
        "Storage Blob Data Reader": {
            "step": "4.2.4",
            "description": "assigne the role Storage Blob Data Reader to the UAI on the Azure Storage account.",
        },
    }

##### 4.2.2 Grant the user identity access to the Azure Content Safety resource
Cognitive Services User role is required to access the Azure Content Safety resource.

In [ ]:
if aacs_authentication_mode == "UAI":
    role_name = "Cognitive Services Contributor"  # minimum role required for listing AACS access keys
    scope = aacs_resource_id

    role_defs = role_definition_client.role_definitions.list(scope=scope)
    role_def = next((r for r in role_defs if r.role_name == role_name))

    from azure.core.exceptions import ResourceExistsError

    try:
        role_assignment_client.role_assignments.create(
            scope=scope,
            role_assignment_name=str(uuid.uuid4()),
            parameters=RoleAssignmentCreateParameters(
                role_definition_id=role_def.id,
                principal_id=uai_principal_id,
                principal_type="ServicePrincipal",
            ),
        )
    except ResourceExistsError as ex:
        pass
    except Exception as ex:
        print(ex)
        raise ex

    if role_name in uai_role_check_list:
        del uai_role_check_list[role_name]
    print(
        f"Role assignment for {role_name} at the Azure AI Content Safety resource level completed."
    )

##### 4.2.3 Assign AcrPull at the workspace container registry scope
Since we will create the content safety enabled LlaMa endpoint with User Assigned Identity, the user managed identity must have Storage Blob Data Reader permission on the storage account for the workspace, and AcrPull permission on the Azure Container Registry (ACR) for the workspace. Make sure your User Assigned Identity has the right permission.

In [ ]:
if aacs_authentication_mode == "UAI":
    workspace = ml_client.workspaces.get(workspace_name)
    container_registry = workspace.container_registry

    role_name = "AcrPull"

    role_defs = role_definition_client.role_definitions.list(scope=container_registry)
    role_def = next((r for r in role_defs if r.role_name == role_name))

    from azure.core.exceptions import ResourceExistsError

    try:
        role_assignment_client.role_assignments.create(
            scope=container_registry,
            role_assignment_name=str(uuid.uuid4()),
            parameters=RoleAssignmentCreateParameters(
                role_definition_id=role_def.id,
                principal_id=uai_principal_id,
                principal_type="ServicePrincipal",
            ),
        )
    except ResourceExistsError as ex:
        pass
    except Exception as ex:
        print(ex)
        raise ex

    if role_name in uai_role_check_list:
        del uai_role_check_list[role_name]
    print("Role assignment for AcrPull at the workspace container registry completed.")

##### 4.2.4 Assign Storage Blob Data Reader at the workspace storage account scope

In [ ]:
if aacs_authentication_mode == "UAI":
    role_name = "Storage Blob Data Reader"
    blob_scope = workspace.storage_account

    role_defs = role_definition_client.role_definitions.list(scope=blob_scope)
    role_def = next((r for r in role_defs if r.role_name == role_name))

    from azure.core.exceptions import ResourceExistsError

    try:
        role_assignment_client.role_assignments.create(
            scope=blob_scope,
            role_assignment_name=str(uuid.uuid4()),
            parameters=RoleAssignmentCreateParameters(
                role_definition_id=role_def.id,
                principal_id=uai_principal_id,
                principal_type="ServicePrincipal",
            ),
        )
    except ResourceExistsError as ex:
        pass
    except Exception as ex:
        print(ex)
        raise ex

    if role_name in uai_role_check_list:
        del uai_role_check_list[role_name]
    print(
        "Role assignment for `Storage Blob Data Reader` at the workspace storage account completed."
    )

### 5. Create Content Safety enabled LLaMA online endpoint

#### 5.1 Decide on SKU and instance count for the Content Safety enabled LLaMA online endpoint.
TODO: Add more details about SKU and instance count recommandation.

In [ ]:
compute_sku_for_llama = "Standard_DS5_v2"
compute_instance_count_for_llama = 1

safety_llama_auth_mode = "key"  # currently, "key" and "aml_token" are supported
print(
    f"Will create LLaMA endpoint {endpoint_name} using {compute_instance_count_for_llama} {compute_sku_for_llama} compute instance(s)"
)

#### 5.2 Create the Safety-Enabled LLaMA Online Endpoint
This step may take a few minutes.

__Before we proceed to the next step, let's make sure we didn't miss anything in the previous steps, please execute the following script to check on that:__

In [ ]:
# Check everything is properly done before creating the Azure AI Content Safety Enabled LLaMA online endpoint
missing_steps = []
if aacs_authentication_mode == "UAI":
    print(
        "You selected UAI to deploy the Azure AI Content Safety Enabled LLaMA online endpoint, checking if the UAI has the required roles assigned..."
    )
    if uai_role_check_list:
        for key, value in uai_role_check_list.items():
            missing_steps.append(
                f'Please go to step {value["step"]} to {value["description"]}'
            )

if missing_steps:
    print("Seems you missed some step above.")
    steps = "\n".join(missing_steps)
    raise Exception(f"Please complete the missing steps before proceeding:\n{steps}")
else:
    print(
        "All steps are completed, proceeding to create the Azure AI Content Safety Enabled LLaMA online endpoint..."
    )

#### 5.3 Check the latest version of LLaMA model

In [ ]:
available_llama_models_pre_trained = ["Llama-2-7b", "Llama-2-13b"]
available_llama_models_fine_tuned = ["Llama-2-7b-chat", "Llama-2-13b-chat"]

model_name = "gpt2"  # TODO(mingtwan) change to LLaMA

registry_ml_client = MLClient(credential, registry_name="azureml")
version_list = list(
    registry_ml_client.models.list(model_name)
)  # list available versions of the model
foundation_model = None
if len(version_list) == 0:
    print("Model not found in registry")
else:
    model_version = version_list[0].version
    foundation_model = registry_ml_client.models.get(model_name, model_version)
    print(
        f"Using model name: {foundation_model.name}, version: {foundation_model.version}, id: {foundation_model.id} for inferencing"
    )

#### 5.4 Create environment for LLaMA endpoint

In [ ]:
from azure.ai.ml.entities import Environment, BuildContext

try:
    env = ml_client.environments.get("LLaMA-ENVIRONMENT", label="latest")
    print("---Environment already exists---")
except:
    print("---Creating environment---")
    env = Environment(name="LLaMA-ENVIRONMENT", build=BuildContext(path="./docker_env"))
    ml_client.environments.create_or_update(env)


# TODO: Add a check to see if the job finished

#### 5.5 Create the online endpoint

In [ ]:
# environment variables that will be used in the scoring script
# environment variable names
env_key_of_aacs_endpoint = "CONTENT_SAFETY_ENDPOINT"
env_key_of_aacs_account_name = "CONTENT_SAFETY_ACCOUNT_NAME"
env_key_of_aacs_key = "CONTENT_SAFETY_KEY"  # if the UAI_CLIENT_ID is not provided, the the script will fallback to use the access key of the AACS resource.

env_key_of_uai_id = "UAI_CLIENT_ID"  # if provided, the script will use the UAI's AAD token to obtain the access key of the LLaMA online endpoint, and use the token to authenticate to the AACS resource directly.
env_key_of_subscription_id = "SUBSCRIPTION_ID"
env_key_of_resource_group_name = "RESOURCE_GROUP_NAME"
env_key_of_workspace_name = "WORKSPACE_NAME"

from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    BatchEndpoint,
    ManagedOnlineEndpoint,
    CodeConfiguration,
    Environment,
    ManagedIdentityConfiguration,
    IdentityConfiguration,
)

if not aacs_endpoint:
    raise Exception("AACS Endpoint is not valid.")
else:
    print(f"AACS Endpoint: {aacs_endpoint}")

environment_variables = {
    env_key_of_aacs_endpoint: aacs_endpoint,
    env_key_of_aacs_account_name: aacs_name,
    env_key_of_subscription_id: subscription_id,
    env_key_of_resource_group_name: resource_group_name,
    env_key_of_workspace_name: workspace_name,
}

uai_identity_config = None
if aacs_authentication_mode == "UAI":
    environment_variables[env_key_of_uai_id] = uai_client_id
    uai_identity_config = IdentityConfiguration(
        type="user_assigned",
        user_assigned_identities=[ManagedIdentityConfiguration(resource_id=uai_id)],
    )
else:
    print(
        "UAI_CLIENT_ID is not provided. The script will fallback to use the access key of the AACS resource and the LLaMA online endpoint."
    )
    # use environment varibale to pass access keys
    environment_variables[env_key_of_aacs_key] = aacs_access_key

deployment_name = "blue"
deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=foundation_model.id,
    code_configuration=CodeConfiguration(code="./score", scoring_script="score.py"),
    environment=env,
    instance_type=compute_sku_for_llama,
    instance_count=compute_instance_count_for_llama,
    environment_variables=environment_variables,
)


endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Azure AI Content Safety enabled LLaMA online endpoint",
    auth_mode=safety_llama_auth_mode,
    identity=uai_identity_config,
)

batch_endpoint = BatchEndpoint(
    name=endpoint_name,
    description="Azure AI Content Safety enabled LLaMA batch endpoint",
    auth_mode=safety_llama_auth_mode,
    identity=uai_identity_config,
)

# create online endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

endpoint = ml_client.online_endpoints.get(endpoint_name)
print(endpoint.identity.type)
print(endpoint.identity.user_assigned_identities)

# create deployment
ml_client.online_deployments.begin_create_or_update(deployment).result()
# check status
deployment = ml_client.online_deployments.get(
    endpoint_name=endpoint_name, name=deployment.name
)
print(deployment)
# Set traffic to 100% for deployment
endpoint.traffic = {str(deployment.name): 100}
ml_client.begin_create_or_update(endpoint).result()

### 6. Test the Azure AI Content Safety Enabled LLaMA online endpoint.

#### 6.1 Test endpoint with normal sample request

In [ ]:
import os

test_src_dir = "./safety-llama-test"
os.makedirs(test_src_dir, exist_ok=True)
print(f"test script directory: {test_src_dir}")

In [ ]:
%%writefile {test_src_dir}/sample-request.json
{
    "input_data": {
          "columns": ["input_string"],
          "index": [0],
          "data": [ "My name is John and I " ]
        }
}

In [ ]:
sample_data = f"{test_src_dir}/sample-request.json"
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    request_file=sample_data,
)

#### 6.2 Test endpoint with harmful sample request

In [ ]:
%%writefile {test_src_dir}/sample-harmful-request.json
{
    "input_data": {
          "columns": ["input_string"],
          "index": [0],
          "data": [ "I wanna kill you" ]
        }
}

In [ ]:
sample_data = f"{test_src_dir}/sample-harmful-request.json"
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    request_file=sample_data,
)